In [1]:
# import necessary library
import pandas as pd

In [2]:
# import the data 
data = pd.read_excel('data_goodreads.xlsx')
data.head()

,user_id,gender,ratings_count,reviews_count,year_joined,young_age,location,friends,fav_genres,groups_count,2018,avg_page_18,2019,avg_page_19,2020,avg_page_20
0,1,F,806,34,2014,1,Asia,103,"Chick-lit, Christian, Ebooks, Fantasy, Fiction...",6,0,209,0,199,1,297
1,2,F,822,90,2013,0,USA,225,"Chick-lit, Contemporary, Fantasy, Fiction, His...",0,1,388,1,383,1,355
2,3,F,114,4,2014,1,Asia,7,"Chick-lit, Contemporary, Ebooks, Romance, Self...",1,1,288,1,292,0,284
3,4,F,1265,630,2015,1,Asia,588,"Fantasy, Historical fiction, Manga, and Young-...",1,1,302,1,346,1,380
4,5,F,380,1,2014,0,EU,56,"Comics, Fantasy, Historical fiction, Humor and...",1,1,327,1,628,0,278


In [3]:
# checking the shape of our data
# 100 rows and 16 columns
data.shape

(100, 16)

In [4]:
# quick overview of what is inside each column
data.describe()

,user_id,ratings_count,reviews_count,year_joined,young_age,friends,groups_count,2018,avg_page_18,2019,avg_page_19,2020,avg_page_20
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,50.500000,419.580000,146.970000,2014.050000,0.620000,338.620000,3.610000,0.510000,315.340000,0.600000,319.540000,0.640000,301.220000
std,29.011492,495.463607,215.218398,2.851103,0.487832,460.103892,5.298933,0.502418,78.186157,0.492366,105.844886,0.482418,64.801418
min,1.000000,13.000000,0.000000,2007.000000,0.000000,6.000000,0.000000,0.000000,156.000000,0.000000,107.000000,0.000000,157.000000
25%,25.750000,135.250000,5.750000,2012.000000,0.000000,63.250000,0.000000,0.000000,270.000000,0.000000,262.750000,0.000000,254.500000
50%,50.500000,289.500000,49.000000,2014.000000,1.000000,162.000000,2.000000,1.000000,302.500000,1.000000,309.000000,1.000000,302.000000
75%,75.250000,529.000000,154.000000,2016.000000,1.000000,435.250000,4.250000,1.000000,362.250000,1.000000,345.250000,1.000000,347.500000
max,100.000000,3174.000000,1034.000000,2019.000000,1.000000,2389.000000,26.000000,1.000000,527.000000,1.000000,866.000000,1.000000,453.000000


In [5]:
# checking for nulls
# there's no nulls 
data.isnull().sum()

user_id          0
gender           0
ratings_count    0
reviews_count    0
year_joined      0
young_age        0
location         0
friends          0
fav_genres       0
groups_count     0
2018             0
avg_page_18      0
2019             0
avg_page_19      0
2020             0
avg_page_20      0
dtype: int64

In [6]:
# checking and make sure the data types are correct
data.dtypes

user_id           int64
gender           object
ratings_count     int64
reviews_count     int64
year_joined       int64
young_age         int64
location         object
friends           int64
fav_genres       object
groups_count      int64
2018              int64
avg_page_18       int64
2019              int64
avg_page_19       int64
2020              int64
avg_page_20       int64
dtype: object

## Data cleaning

In [7]:
# seeing how many unique genres we have
unique_genre = []
for li in data['fav_genres']:
    for genre in li.split(','):
        unique_genre.append(genre)

print(len(set(unique_genre)))
print(set(unique_genre))
# need to clean the genres and grouped similar genres together

125
{'\xa0Chick-lit', ' and\xa0Psychology', '\xa0Biography', '\xa0historical-romance', '\xa0short-stories', ' and\xa0young adult fantasy', ' and\xa0Self help', '\xa0Philosophy', ' Nonfiction', " and\xa0Women's Fiction", ' and\xa0Humor and Comedy', '\xa0Spirituality', ' and\xa0fairy-tales', '\xa0Science fiction', 'Adult Fiction', '\xa0regency', '\xa0Memoir', ' and\xa0Non-fiction', 'Art', '\xa0fantasy-romance', '\xa0Music', '\xa0Young-adult', '\xa0Christian', ' and\xa0Sports', '\xa0Business', '\xa0time-travel', ' and\xa0dark-fantasy', '\xa0fairy-tales', ' and\xa0Travel', '\xa0Crime', '\xa0urban-fantasy', '\xa0Religion', ' Self Help', '\xa0scotland', '\xa0Horror', '\xa0Politics', 'Classics', '\xa0vampires', '\xa0Mystery', 'Chick-lit', 'Fiction', '\xa0omegaverse', ' and\xa0Poetry', ' and\xa0short-stories', '\xa0time-travel-romance', '\xa0War', ' and\xa0young-adult-romance', ' Non-fiction', ' and\xa0Spirituality', '\xa0historical', ' and\xa0Suspense', ' and\xa0Memoir', '\xa0Book Club', '\xa

In [8]:
# clean the fav_genres column 
for row in range(len(data['fav_genres'])):
    data['fav_genres'][row] = data['fav_genres'][row].replace('\xa0','').replace(' and','').split(',')

C:\Users\glady\AppData\Local\Temp/ipykernel_3848/468483999.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['fav_genres'][row] = data['fav_genres'][row].replace('\xa0','').replace(' and','').split(',')


In [9]:
# function to clean the genres fiction
# as there are many types of fiction, it's better to put it just as fiction 
def clean_fiction(li):
    new = []
    for text in li:
        l = text.split(' ')
        if len(l) > 1:
            if l[1].lower() == 'fiction':
                new.append('Fiction')
            else:
                new.append(' '.join(l).strip())
        else:
            new.append(' '.join(l).strip())

    return new

In [10]:
# apply the above function and make a new column of it 
data['fav_genres_clean'] = data['fav_genres'].apply(clean_fiction)

In [11]:
data['fav_genres_clean'][0]

['Chick-lit',
 'Christian',
 'Ebooks',
 'Fantasy',
 'Fiction',
 'Poetry',
 'Romance',
 'Self help',
 'Young-adult']

In [12]:
# clean the genres and to make the population smaller, so we can categorize preferences later
def clean_genres(li):
    new = []
    for l in li:
        if l.lower() == 'ebooks' or l.lower() == 'cookbooks' or l.lower() == 'travel' or l.lower() == 'humor comedy'or l.lower() == 'book club' or l.lower() == 'gothic' or l.lower() == 'sports' or l.lower() == 'lgbt' or l.lower() == 'gay lesbian' or l.lower() == 'scotland' or l.lower() == 'new-adult' or l.lower() == 'disability' or l.lower() == 'short-stories' or l.lower() == 'feminism' or l.lower() == 'football' or l.lower() == 'omegaverse' or l.lower() == 'asian-literature':
            pass
        elif l.lower() == 'suspense' or l.lower() == 'crime' or l.lower() == 'mystery':
            new.append('Thriller')
        elif l.lower() == 'paranormal':
            new.append('Horror')
        elif l.lower() == 'personal-development' or l.lower() == 'self help':
            new.append('Self help')
        elif l.lower() == 'chick-lit' or l.lower() == 'paranormal-romance' or l.lower() == 'historical-romance' or l.lower() == 'fantasy-romance' or l.lower() == 'time-travel-romance':
            new.append('Romance')
        elif l.lower() == 'contemporary' or l.lower() == 'vampires'  or l.lower() == 'magic' or l.lower() == 'time-travel' or l.lower() == 'mythology' or l.lower() == 'fairy-tales':
            new.append('Fiction')
        elif l.lower() == 'manga' or l.lower() == 'graphic novels':
            new.append('Comics')
        elif l.lower() == 'nonfiction':
            new.append('Non-fiction')
        elif l.lower() == 'leadership':
            new.append('Business')
        elif l.lower() == 'memoir':
            new.append('Biography')
        elif l.lower() == 'christian' or l.lower() == 'spirituality':
            new.append('Religion')
        elif l.lower() == 'music' or l.lower() == 'poetry':
            new.append('Art')
        elif l.lower() == 'regency' or l.lower() == 'medieval' or l.lower() == 'military history' or l.lower() == 'war' or l.lower() == 'historical':
            new.append('History')
        elif l.lower() == 'classic-literature':
            new.append('Classics')
        elif l.lower() == 'youn-adult-paranormal-romance' or l.lower() == 'young-adult-romance' or l.lower() == 'young adult romance' or l.lower() == 'young adult fantasy' or l.lower() == 'young adult':
            new.append('Young-adult')
        elif l.lower() == 'fantasy' or l.lower() == 'dark-fantasy' or l.lower() == 'high-fantasy' or l.lower() == 'epic-fantasy'  or l.lower() == 'urban-fantasy':
            new.append('Fantasy')
        elif l.lower() == 'childrens':
            new.append("Children's")

        else:
            new.append(l)

    return new

In [13]:
# apply above function 
data['fav_genres_clean'] = data['fav_genres_clean'].apply(clean_genres)


In [14]:
unique_genre = []
for li in data['fav_genres_clean']:
    for genre in li:
        unique_genre.append(genre)

print(len(set(unique_genre)))
print(set(unique_genre))

21
{'Business', 'Young-adult', 'Comics', 'Classics', 'Psychology', 'Horror', 'Non-fiction', 'Religion', 'Fiction', 'Philosophy', 'Fantasy', 'Biography', 'Art', 'History', 'Self help', "Children's", 'Romance', 'Science', 'historical-mystery', 'Thriller', 'Politics'}


In [15]:
# categorize genres whether it is fiction or nonfiction  
def categorize(li):
    new = []
    for l in li:
        if l == 'Comics' or l == 'Fantasy' or l == 'Horror' or l == 'Romance' or l == 'Young-adult' or l == 'Thriller' or l == 'Religion' or l == 'Classics' :
            new.append('Fiction')
        elif l == 'Politics' or l == 'Self help' or l == 'Philosophy' or l == 'History' or l == 'Biography' or l == 'Science' or l == 'Psychology' or l == 'Business':
            new.append('Non-fiction')
        else:
            new.append(l)
    
    return new 

In [16]:
# apply above function and make a new column
data['category'] = data['fav_genres_clean'].apply(categorize)

In [17]:
# remove all other genres that is not included in fiction or non-fiction
# as there are many niche genres 
def fic_nonfic(li):
    new = []
    for l in li:
        if l.lower() == 'fiction' or l.lower() == 'non-fiction':
            new.append(l)
        else:
            pass
        
    return new

In [18]:
# apply above function
data['category'] = data['category'].apply(fic_nonfic)

In [19]:
# # copy the column 'category'
data['category_simplified'] = data['category'].copy()

In [20]:
# make the 'category' column into a set so we only have either fiction, non-fiction, or both
data['category'] = data['category'].apply(set)
data['category'] = data['category'].apply(str).str.strip('{}').str.strip("'")
data['category'] = data['category'].str.replace("', '", ', ')

In [21]:
dumm = pd.get_dummies(data['category'])
dumm.rename(columns={"Non-fiction', 'Fiction" : 'Both'}, inplace=True)

In [22]:
## merge the dummies to the original dataset

data_clean = pd.merge(left= data, 
                                right= dumm,
                                left_index = True,
                                right_index= True)

data_clean.head()

,user_id,gender,ratings_count,reviews_count,year_joined,young_age,location,friends,fav_genres,groups_count,...,2019,avg_page_19,2020,avg_page_20,fav_genres_clean,category,category_simplified,Fiction,"Fiction, Non-fiction",Non-fiction
0,1,F,806,34,2014,1,Asia,103,"[Chick-lit, Christian, Ebooks, Fantasy, Fictio...",6,...,0,199,1,297,"[Romance, Religion, Fantasy, Fiction, Art, Rom...","Fiction, Non-fiction","[Fiction, Fiction, Fiction, Fiction, Fiction, ...",0,1,0
1,2,F,822,90,2013,0,USA,225,"[Chick-lit, Contemporary, Fantasy, Fiction, Hi...",0,...,1,383,1,355,"[Romance, Fiction, Fantasy, Fiction, Fiction, ...","Fiction, Non-fiction","[Fiction, Fiction, Fiction, Fiction, Fiction, ...",0,1,0
2,3,F,114,4,2014,1,Asia,7,"[Chick-lit, Contemporary, Ebooks, Romance, Sel...",1,...,1,292,0,284,"[Romance, Fiction, Romance, Self help, Young-a...","Fiction, Non-fiction","[Fiction, Fiction, Fiction, Non-fiction, Ficti...",0,1,0
3,4,F,1265,630,2015,1,Asia,588,"[Fantasy, Historical fiction, Manga, Young-adult]",1,...,1,346,1,380,"[Fantasy, Fiction, Comics, Young-adult]",Fiction,"[Fiction, Fiction, Fiction, Fiction]",1,0,0
4,5,F,380,1,2014,0,EU,56,"[Comics, Fantasy, Historical fiction, Humor Co...",1,...,1,628,0,278,"[Comics, Fantasy, Fiction, Art, Psychology, Sc...","Fiction, Non-fiction","[Fiction, Fiction, Fiction, Non-fiction, Non-f...",0,1,0


In [23]:
# function to make a boolean value whether people prefer fiction (1) or nonfiction (0)
# based on whether they prefer more fiction genres or non-fiction genres
def simplifying(li):
    for row in li:
        if row.count('Fiction') > row.count('Non-fiction'):
            return('1')
        else:
            return('0')
    

In [24]:
# # apply above function 
data_clean['category_simplified'] = data_clean['category_simplified'].apply(simplifying)

In [25]:
# function to make the 'gender' column into boolean value
def gender_category(li):
    for row in li:
        if row == 'F':
            return('1')
        else:
            return('0')

In [26]:
# apply above function
data_clean['gender'] = data['gender'].apply(gender_category)

In [27]:
# function to sum the total amount of challenge completed(0-3)
def total_page (row):
    x = row['avg_page_18'] + row['avg_page_19'] + row['avg_page_20']
    return x

# function to sum the total amount of avg page from 2018 to 2020
def challenge_completed (row):
    x = row[2018] + row[2019] + row[2020]
    return x

In [28]:
# apply above functions and make a new column 
data_clean['total_page'] = data.apply(total_page, axis=1)
data_clean['challenge_completed'] = data.apply(challenge_completed, axis=1)
data_clean.head()

,user_id,gender,ratings_count,reviews_count,year_joined,young_age,location,friends,fav_genres,groups_count,...,2020,avg_page_20,fav_genres_clean,category,category_simplified,Fiction,"Fiction, Non-fiction",Non-fiction,total_page,challenge_completed
0,1,1,806,34,2014,1,Asia,103,"[Chick-lit, Christian, Ebooks, Fantasy, Fictio...",6,...,1,297,"[Romance, Religion, Fantasy, Fiction, Art, Rom...","Fiction, Non-fiction",1,0,1,0,705,1
1,2,1,822,90,2013,0,USA,225,"[Chick-lit, Contemporary, Fantasy, Fiction, Hi...",0,...,1,355,"[Romance, Fiction, Fantasy, Fiction, Fiction, ...","Fiction, Non-fiction",1,0,1,0,1126,3
2,3,1,114,4,2014,1,Asia,7,"[Chick-lit, Contemporary, Ebooks, Romance, Sel...",1,...,0,284,"[Romance, Fiction, Romance, Self help, Young-a...","Fiction, Non-fiction",1,0,1,0,864,2
3,4,1,1265,630,2015,1,Asia,588,"[Fantasy, Historical fiction, Manga, Young-adult]",1,...,1,380,"[Fantasy, Fiction, Comics, Young-adult]",Fiction,1,1,0,0,1028,3
4,5,1,380,1,2014,0,EU,56,"[Comics, Fantasy, Historical fiction, Humor Co...",1,...,0,278,"[Comics, Fantasy, Fiction, Art, Psychology, Sc...","Fiction, Non-fiction",1,0,1,0,1233,2


In [29]:
# create a csv 
data_clean.to_csv('goodreads_clean_final.csv', index=False)